In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV

In [2]:
def metricas(model,Xt,Xv,yt,yv):
    print "ROC train:%.3f | ROC test:%.3f " %(roc_auc_score(y_true=yt,y_score=model.predict_proba(Xt)[:,1]),roc_auc_score(y_true=yv,y_score=model.predict_proba(Xv)[:,1]))
    print "ACC train:%.3f | ACC test:%.3f " %(accuracy_score(y_true=yt,y_pred=model.predict(Xt)),accuracy_score(y_true=yv,y_pred=model.predict(Xv)))

In [3]:
df = pickle.load(open('churn_con_woe','rb'))

In [4]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,...,W_V_ACCT_LGHT,W_V_N_VM_MESS,W_V_T_DAY_CHG,W_V_T_EVE_CALLS,W_V_T_EVE_CHG,W_V_T_NIG_CALLS,W_V_T_NIG_CHG,W_V_T_INT_CALLS,W_V_T_INT_CHG,W_V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,0.061033,1.219037,-1.864202,0.068283,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
1,156,No,No,0,29.67,116,16.78,86,10.73,2,...,0.061033,-0.147174,0.561314,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
2,57,No,No,0,14.60,127,16.48,93,10.42,2,...,0.053118,-0.147174,0.178109,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
3,58,No,Yes,20,33.07,89,18.16,92,10.65,2,...,0.053118,1.219037,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
4,149,No,No,0,31.89,81,17.12,79,11.89,1,...,0.061033,-0.147174,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956


In [5]:
X = df[[v for v in df.columns if v[:2]=='V_']+[v for v in df.columns if v[:4]=='W_C_']].copy()
y = df['TARGET'].copy()

In [6]:
pipe = make_pipeline(MinMaxScaler())
pipe.fit(X)
Xs = pd.DataFrame(pipe.transform(X),columns=X.columns)

In [7]:
Xs.head()

,V_ACCT_LGHT,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,W_C_IP,W_C_VMP
0,0.633880,0.581395,0.841885,0.489362,0.492640,0.377778,0.693383,0.166667,0.500000,0.166667,1.0,1.0
1,0.786885,0.000000,0.484328,0.670213,0.492640,0.322222,0.667288,0.083333,0.544199,0.000000,1.0,0.0
2,0.245902,0.000000,0.134432,0.787234,0.477920,0.400000,0.638397,0.083333,0.508287,0.000000,1.0,0.0
3,0.251366,0.465116,0.563269,0.382979,0.560353,0.388889,0.659832,0.083333,0.464088,0.166667,1.0,1.0
4,0.748634,0.000000,0.535872,0.297872,0.509323,0.244444,0.775396,0.000000,0.411602,0.166667,1.0,0.0


In [8]:
Xt,Xv,yt,yv = train_test_split(Xs,y,train_size=0.7)

In [9]:
model = KNeighborsClassifier()

In [10]:
model.fit(Xt,yt)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [11]:
metricas(model,Xt,Xv,yt,yv)

ROC train:0.959 | ROC test:0.819 
ACC train:0.921 | ACC test:0.897 


In [12]:
param_grid = dict(n_neighbors = range(4,90)
)

In [13]:
grid = GridSearchCV(cv=3,
                  verbose=True,
                  scoring='roc_auc',
                  estimator=model,
                  n_jobs=-1,
                  param_grid=param_grid)

In [14]:
grid.fit(Xs,y)

Fitting 3 folds for each of 86 candidates, totalling 258 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 258 out of 258 | elapsed:   17.3s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=True)

In [16]:
model = grid.best_estimator_
model.fit(Xt,yt)

metricas(model,Xt,Xv,yt,yv)

ROC train:0.895 | ROC test:0.885 
ACC train:0.868 | ACC test:0.873 
